In [ ]:
from sklearn.svm import SVC
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, plot_roc_curve

from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from collections import Counter

from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay


import matplotlib.pyplot as plt


In [ ]:
dataFile1 = 'F:/Radiomics/MONKEY_trainMI.csv'
data = pd.read_csv(dataFile1)
X_train = data.iloc[: ,2:]
y_train = data['DIS']



dataFile2 = 'F:/Radiomics/HUMAN_testMI.csv'
data2 = pd.read_csv(dataFile2)
X_test = data2.iloc[: ,2:]
y_test = data2['DIS']


In [ ]:
 
smo = SMOTE(random_state = 21)
X_train, y_train = smo.fit_resample(X_train, y_train)
Counter(y_train)

In [ ]:
svc = SVC(kernel='rbf',probability=True)
svc.fit(X_train,y_train)

print(svc.score(X_train,y_train))
print(svc.score(X_test,y_test))

In [ ]:
print(classification_report(y_test, svc.predict(X_test),digits=5,))

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, svc.predict(X_test)))

In [ ]:
#ROC

svc_train = RocCurveDisplay.from_estimator(svc, X_train, y_train)
svc_2 = RocCurveDisplay.from_estimator(svc, X_test, y_test, ax=svc_train.ax_)


In [ ]:
#auc
#1
a = roc_auc_score(y_train, svc.predict_proba(X_train)[:, 1])
b = roc_auc_score(y_test, svc.predict_proba(X_test)[:, 1])
#2
c = roc_auc_score(y_train, svc.decision_function(X_train))
d = roc_auc_score(y_test, svc.decision_function(X_test))

#3
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, svc.predict_proba(X_test)[:, 1], pos_label=2)
e = metrics.auc(fpr, tpr)

print(a)
print(b)
print(c)
print(d)
print(e)

In [ ]:
#train
from sklearn.metrics import roc_auc_score

def bootstrap_auc(svc, X_train, y_train, X_test, y_test, nsamples=1000):
    auc_values = []
    for b in range(nsamples):
        idx = np.random.randint(X_train.shape[0], size=X_train.shape[0])
        svc.fit(X_train.iloc[idx], y_train.iloc[idx])
        pred = svc.predict_proba(X_train)[:, 1]
        roc_auc = roc_auc_score(y_train.ravel(), pred.ravel())
        auc_values.append(roc_auc)
    return np.percentile(auc_values, (2.5, 97.5))

bootstrap_auc(svc, X_train, y_train, X_test, y_test, nsamples=1000)

In [ ]:
#test
from sklearn.metrics import roc_auc_score

def bootstrap_auc(svc, X_train, y_train, X_test, y_test, nsamples=1000):
    auc_values = []
    for b in range(nsamples):
        idx = np.random.randint(X_train.shape[0], size=X_train.shape[0])
        svc.fit(X_train.iloc[idx], y_train.iloc[idx])
        pred = svc.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test.ravel(), pred.ravel())
        auc_values.append(roc_auc)
    return np.percentile(auc_values, (2.5, 97.5))

bootstrap_auc(svc, X_train, y_train, X_test, y_test, nsamples=1000)